# Ordenação de arrays em C#

C# oferece algumas maneiras simples de ordenar arrays. Na realidade, os métodos disponíveis pelas classes do .NET Framework facilitam muito essa tarefa, mas escondem a complexidade por trás dos algoritmos de ordenação.

Por exemplo, consideremos um array de inteiros que queremos ordenar em ordem crescente.

In [1]:
int[] array = [1,4,5,8,9,2,6,7,9,-1];

Para mostrar o array, definimos uma função `PrintArray` que itera sobre os elementos do array e os imprime no terminal.

In [2]:
void PrintArray(int[] a) {
    foreach(int i in array){
        Console.Write($"{i} ");
    }
}
PrintArray(array);

1 4 5 8 9 2 6 7 9 -1 

## Bubble Sort

Um dos algoritmos de ordenação mais simples é o Bubble Sort. A ideia básica do Bubble Sort é percorrer o array várias vezes, comparando elementos adjacentes e trocando-os de posição se estiverem na ordem errada. Esse processo é repetido até que o array esteja completamente ordenado.

<p align="center">
<img src="figures/bubble_sort.svg"/>
</p>
Em C#, temos:

In [3]:
void BubbleSort(int[] array) {
    int n = array.Length;
    for (int i = 0; i < n - 1; i++) {
        for (int j = 0; j < n - i - 1; j++) {
            if (array[j] > array[j + 1]) {
                int temp = array[j];
                array[j] = array[j + 1];
                array[j + 1] = temp;
            }
        }
    }
}
BubbleSort(array);
PrintArray(array);

-1 1 2 4 5 6 7 8 9 9 

## Abstração do critério de comparação

O nosso algoritmo de ordenação funciona corretamente para ordenar arrays de inteiros em ordem crescente. No entanto, e se quisermos ordenar em ordem decrescente?

Seria necessário modificar o código do algoritmo. Conseguimos isolar as linhas de código relevantes:

```csharp
if (array[j] > array[j + 1]) {
// Troca posição dos elementos j e j+1
}
```

Aqui, o critério de comparação é `array[j] > array[j + 1]`. Para ordenar em ordem decrescente, precisaríamos alterar essa linha para `array[j] < array[j + 1]`.

Para evitar modificar o código do algoritmo, podemos abstrair o critério de comparação usando a referência de funções.

### Referência de funções

Em C#, podemos passar funções como parâmetros. Para tal, usamos o tipo `Func<...>`, onde os parâmetros genéricos representam os tipos dos argumentos e o tipo de retorno da função.

O tipo `Func` recebe um conjunto de tipos genéricos, onde todos, exceto o último, representam os tipos dos argumentos da função, e o último tipo representa o tipo de retorno da função.

In [4]:
int f1(int a) {
    return a + 2; 
}

void g1(int a, Func<int, int> t){
    int r = t(a);
    Console.WriteLine(r);
}

g1(3, f1);

5


O encadeamento de invocações de funções pode ser representado da seguinte forma:

<p align="center">
<img src="figures/func_example.svg"/>
</p>

Caso o retorno seja `void`, usamos o tipo `Action` em vez de `Func`.

In [5]:
void f2(int a) {
    Console.WriteLine(a);
}

void g2(int a, Action<int> t){
    t(a);
}

g2(3, f2);

3


### Regressando ao Bubble Sort

Recorrendo ao instrumento de referência de funções, podemos definir uma função de comparação que recebe dois inteiros e retorna um valor booleano indicando se o primeiro inteiro deve vir antes do segundo na ordenação.

In [6]:
bool IsGreaterThan(int a, int b) {
    return a > b;
}

bool IsLesserThan(int a, int b) {
    return a < b;
}

// Func<int, int, bool>
bool Dummy(int a, int b) {
    return true;
}

Agora podemos modificar o nosso algoritmo de ordenação para aceitar uma função de comparação como parâmetro.

In [7]:
void BetterBubbleSort(int[] array, Func<int, int, bool> CompareInts) {
    int n = array.Length;
    for (int i = 0; i < n - 1; i++) {
        for (int j = 0; j < n - i - 1; j++) {
            if (CompareInts(array[j], array[j + 1])) {
                int temp = array[j];
                array[j] = array[j + 1];
                array[j + 1] = temp;
            }
        }
    }
}

A mesma implementação base funciona para ordenar em ordem crescente...

In [8]:
bool IsGreaterThan(int a, int b) {
    return a > b;
}
BetterBubbleSort(array, IsGreaterThan);

e em ordem decrescente, dependendo da função de comparação passada como argumento.

In [9]:
BetterBubbleSort(array, IsLesserThan);
PrintArray(array)

9 9 8 7 6 5 4 2 1 -1 

### Funções anónimas

A utilização dos tipos `Func` e `Action` permite abstrair parte do comportamento do código, tornando-o mais flexível e reutilizável. São instrumentos muito úteis, mas implicam a definição de funções auxiliares, para serem chamadas como argumentos.

Para facilitar a utilização destes mecanismos, C# oferece a possibilidade de definir funções anónimas (lambdas) diretamente na chamada do método.

Estas funções anónimas podem ser definidas usando a sintaxe `(param1, param2, ...) => expression` para expressões simples, ou `(param1, param2, ...) => { statements }` para blocos de código mais complexos.

Por exemplo, podemos chamar o nosso algoritmo de ordenação usando funções anónimas para definir os critérios de comparação diretamente na chamada do método:

In [10]:
BetterBubbleSort(array, (a,b) => a > b);
PrintArray(array);

-1 1 2 4 5 6 7 8 9 9 

In [11]:
BetterBubbleSort(array, (a,b) => a < b);
PrintArray(array);

9 9 8 7 6 5 4 2 1 -1 

## Ordenações de coleções de tipos complexos

Podemos usar o mesmo princípio para ordenar coleções de tipos complexos, como objetos de classes definidas pelo utilizador. Basta definir uma função de comparação que compare os atributos relevantes dos objetos.

Vamos considerar o tipo `Pessoa`, com os atributos `Nome` e `Idade`. Podemos ordenar uma lista de objetos `Pessoa` por idade ou por nome, dependendo da função de comparação que passamos para o nosso algoritmo de ordenação.


In [12]:
class Pessoa {
    public Pessoa(string Nome, int Idade) {
        this.Nome = Nome;
        this.Idade = Idade;
    }
    public string Nome { get; }
    public int Idade { get; }

    public override string ToString() {
        return $"{Nome} ({Idade})";
    }
}

Note-se que o tipo `Pessoa` inclui uma re-implementação do método `ToString`, permitindo uma representação legível dos objetos ao serem impressos.

Pode parecer estranho considerar que o método `ToString` está a ser re-implementado, mas na realidade todas as classes em C# herdam implicitamente da classe base `Object`, que define o método `ToString`. Ao re-implementar este método, estamos a fornecer uma versão personalizada para a nossa classe `Pessoa`.

<p align="center">
<img src="figures/basic_inheritance.svg"/>
</p>

Podemos agora definir um array de pessoas.

In [18]:
Pessoa[] pessoas = [
    new Pessoa("Steve", 30),
    new Pessoa("Bob", 25),
    new Pessoa("Charlie", 35),
    new Pessoa("Alice", 30),
];

In [19]:
foreach(Pessoa p in pessoas) {
    Console.WriteLine(p);
}

Steve (30)
Bob (25)
Charlie (35)
Alice (30)


Quando tentamos ordenar o array de pessoas com a nossa implementação do Bubble Sort, recebemos um erro de compilação. Isto acontece porque o algoritmo espera um array de inteiros, mas estamos a fornecer um array de objetos `Pessoa`.

In [16]:
BetterBubbleSort(pessoas, (a, b) => a.Idade > b.Idade);

Error: (1,39): error CS1061: 'int' does not contain a definition for 'Idade' and no accessible extension method 'Idade' accepting a first argument of type 'int' could be found (are you missing a using directive or an assembly reference?)
(1,49): error CS1061: 'int' does not contain a definition for 'Idade' and no accessible extension method 'Idade' accepting a first argument of type 'int' could be found (are you missing a using directive or an assembly reference?)

É então necessário generalizar o nosso algoritmo de ordenação para trabalhar com o tipo de dados `Pessoa`.

In [24]:
void PessoaBubbleSort(Pessoa[] array, Func<Pessoa, Pessoa, bool> compare) {
    // TODO: Falta implementar.
}

Para utilizar a nossa implementação do Bubble Sort com o tipo `Pessoa`, precisamos de definir um critério de comparação adequado. Podemos fazer isso usando funções anónimas para comparar os atributos `Idade` ou `Nome` dos objetos `Pessoa`.

Pretendemos então ordenar o array por ordem crescente de idade e, caso a idade seja igual, por ordem lexicográfica de nome.

In [ ]:
PessoaBubbleSort(pessoas, (a, b) => {
    // TODO: Falta implementar.
});

foreach(Pessoa p in pessoas) {
    Console.WriteLine(p);
}

Bob (25)
Alice (30)
Steve (30)
Charlie (35)
